In [18]:
import pandas as pd

# Cargar el dataset ya preparado
df = pd.read_csv("dataset_RN5.csv")

# Verifica columnas
print(df.columns)
df.head()

Index(['PRECTOTCORR', 'T2M_MAX', 'T2M_MIN', 'RH2M', 'WS10M',
       'ALLSKY_SFC_SW_DWN', 'CLIMA_TIPO'],
      dtype='object')


,PRECTOTCORR,T2M_MAX,T2M_MIN,RH2M,WS10M,ALLSKY_SFC_SW_DWN,CLIMA_TIPO
0,2.67,22.79,13.84,91.55,1.26,13.56,1
1,10.93,24.50,14.65,94.30,1.01,11.17,3
2,5.84,26.57,19.78,94.18,1.17,9.82,3
3,1.03,27.19,20.47,91.01,1.05,9.81,1
4,0.25,27.67,20.96,89.50,1.04,12.98,1


In [19]:
# Variables de entrada (todas menos la etiqueta)
X = df.drop(columns=['CLIMA_TIPO']).values

# Etiqueta de salida
y = df['CLIMA_TIPO'].values

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

# One-hot encoding de etiquetas
y_cat = to_categorical(y, num_classes=4)

# Dividir
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, shuffle=True)

# Escalar entradas
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model_clima = Sequential()
model_clima.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model_clima.add(Dropout(0.3))
model_clima.add(Dense(32, activation='relu'))
model_clima.add(Dropout(0.3))
model_clima.add(Dense(16, activation='relu'))
model_clima.add(Dense(4, activation='softmax'))  # 4 clases

model_clima.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_clima.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 4)              │            68 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,124 (12.20 KB)

 Trainable params: 3,124 (12.20 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model_clima.fit(
    X_train_scaled, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test_scaled, y_test),
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/5
11186/11186 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - accuracy: 0.9266 - loss: 0.1862 - val_accuracy: 0.9726 - val_loss: 0.0686
Epoch 2/5
11186/11186 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.9643 - loss: 0.0893 - val_accuracy: 0.9757 - val_loss: 0.0547
Epoch 3/5
11186/11186 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9711 - loss: 0.0723 - val_accuracy: 0.9843 - val_loss: 0.0368
Epoch 4/5
11186/11186 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9750 - loss: 0.0630 - val_accuracy: 0.9877 - val_loss: 0.0329
Epoch 5/5
11186/11186 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.9767 - loss: 0.0579 - val_accuracy: 0.9817 - val_loss: 0.0373


In [17]:
model.save("modelo_clima_mlp.h5")

NameError: name 'model' is not defined

In [11]:
loss, acc = model_clima.evaluate(X_test_scaled, y_test)
print(f"📉 Loss: {loss:.4f}")
print(f"📈 Accuracy: {acc:.2%}")

2797/2797 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9892 - loss: 0.0251
📉 Loss: 0.0251
📈 Accuracy: 98.92%


In [15]:
def predecir_tipo_clima(variables, modelo, scaler):
    entrada = pd.DataFrame([variables])
    entrada_scaled = scaler.transform(entrada.values)
    pred = modelo.predict(entrada_scaled)[0]  # vector de probabilidades

    etiquetas = ['Soleado', 'Nublado', 'Ventoso', 'Lluvioso']
    clase = pred.argmax()

    print(f"🔍 Predicción: {etiquetas[clase]} ({clase})")
    print("\n📊 Probabilidades por clase:")
    for i, prob in enumerate(pred):
        print(f"  {etiquetas[i]} ({i}): {prob:.2%}")

    return clase, etiquetas[clase], pred

In [16]:
# Ejemplo con variables reales
variables = {
    'PRECTOTCORR': 4.04,
    'T2M_MAX': 30.69,
    'T2M_MIN': 23.48,
    'RH2M': 84.96,
    'WS10M': 1.10,
    'ALLSKY_SFC_SW_DWN': 19.36
}

predecir_tipo_clima(variables, model_clima, scaler)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
🔍 Predicción: Nublado (1)

📊 Probabilidades por clase:
  Soleado (0): 0.00%
  Nublado (1): 99.98%
  Ventoso (2): 0.00%
  Lluvioso (3): 0.02%


(np.int64(1),
 'Nublado',
 array([3.7554773e-18, 9.9980301e-01, 3.7563095e-06, 1.9323033e-04],
       dtype=float32))